In [ ]:
from pathlib import Path

from PIL import Image

from src.model import load_blip2, generate_answer

# 1. Загружаем модель
model, processor, device = load_blip2()
print("Device:", device)

# 2. Пути к картинкам
data_dir = Path("data")
image_paths = {
    "cat": data_dir / "cat.jpg",
    "dog": data_dir / "dog.jpg",
    "girl_cat": data_dir / "girl_cat.jpg",
}

for name, p in image_paths.items():
    print(name, "->", p, "| exists:", p.exists())


In [ ]:
from IPython.display import display

def blip2_caption(image_path, prompt: str = "Describe this image in detail."):
    img = Image.open(image_path).convert("RGB")
    display(img.resize((384, 384)))

    text = generate_answer(
        model=model,
        processor=processor,
        image=img,
        prompt=prompt,
        device=device,
        max_new_tokens=64,
    )
    print("\nBLIP-2:", text)


# Примеры
blip2_caption(image_paths["cat"])


In [ ]:
blip2_caption(image_paths["dog"])
blip2_caption(
    image_paths["girl_cat"],
    prompt="Describe the relationship between the person and the cat.",
)


In [ ]:
def blip2_qa(image_path, question: str):
    img = Image.open(image_path).convert("RGB")
    display(img.resize((384, 384)))

    prompt = f"Question: {question} Answer:"
    answer = generate_answer(
        model=model,
        processor=processor,
        image=img,
        prompt=prompt,
        device=device,
        max_new_tokens=32,
    )
    print(f"\nQ: {question}")
    print("A:", answer)


# Примеры вопросов
blip2_qa(image_paths["dog"], "What breed is this dog?")
blip2_qa(image_paths["girl_cat"], "Where is the cat and what is the person doing?")
blip2_qa(image_paths["cat"], "Is this photo taken indoors or outdoors?")


In [ ]:
def chat_step(image, history: str, user_message: str, max_new_tokens: int = 64):
    """
    history: строка с предыдущим диалогом "User: ...\nAssistant: ...\n..."
    user_message: новый вопрос/реплика пользователя
    """
    if history:
        prompt = (
            "You are a helpful assistant that reasons about the image and the conversation.\n"
            "Image is given once; then follow the dialog.\n\n"
            + history
            + f"\nUser: {user_message}\nAssistant:"
        )
    else:
        prompt = (
            "You are a helpful assistant that describes and reasons about the image.\n"
            f"User: {user_message}\nAssistant:"
        )

    answer = generate_answer(
        model=model,
        processor=processor,
        image=image,
        prompt=prompt,
        device=device,
        max_new_tokens=max_new_tokens,
    )

    new_history = history + f"\nUser: {user_message}\nAssistant: {answer}"
    return answer, new_history


# Инициализируем чат
img_chat = Image.open(image_paths["girl_cat"]).convert("RGB")
display(img_chat.resize((384, 384)))

history = ""


In [ ]:
def chat_step(image, history: str, user_message: str, max_new_tokens: int = 64):
    """
    history: строка с предыдущим диалогом "User: ...\nAssistant: ...\n..."
    user_message: новый вопрос/реплика пользователя
    """
    if history:
        prompt = (
            "You are a helpful assistant that reasons about the image and the conversation.\n"
            "Image is given once; then follow the dialog.\n\n"
            + history
            + f"\nUser: {user_message}\nAssistant:"
        )
    else:
        prompt = (
            "You are a helpful assistant that describes and reasons about the image.\n"
            f"User: {user_message}\nAssistant:"
        )

    answer = generate_answer(
        model=model,
        processor=processor,
        image=image,
        prompt=prompt,
        device=device,
        max_new_tokens=max_new_tokens,
    )

    new_history = history + f"\nUser: {user_message}\nAssistant: {answer}"
    return answer, new_history


# Инициализируем чат
img_chat = Image.open(image_paths["girl_cat"]).convert("RGB")
display(img_chat.resize((384, 384)))

history = ""


In [ ]:
# Шаг 1
answer, history = chat_step(
    img_chat,
    history=history,
    user_message="Describe this photo briefly.",
)
print("Assistant:", answer)

# Шаг 2
answer, history = chat_step(
    img_chat,
    history=history,
    user_message="What emotions do you think the person might be feeling?",
)
print("\nAssistant:", answer)

# Шаг 3
answer, history = chat_step(
    img_chat,
    history=history,
    user_message="Invent a short caption for Instagram for this photo.",
)
print("\nAssistant:", answer)

# Посмотреть, что накопилось в истории
print("\n--- FULL HISTORY ---")
print(history)


In [ ]:
img_chat = Image.open(image_paths["girl_cat"]).convert("RGB")
display(img_chat.resize((384, 384)))

history = ""

while True:
    q = input("\nYou: ")
    if not q.strip():
        break
    answer, history = chat_step(img_chat, history, q)
    print("Assistant:", answer)
